In [12]:
import os
os.environ["DEEPLAKE_DOWNLOAD_PATH"] = "./dataset"

In [13]:
import deeplake
from torchvision import transforms

ds = deeplake.dataset('hub://activeloop/wiki-art', access_method = 'local')

transformation = transforms.Compose([
    transforms.ToTensor(),
    # TODO: figure out these values
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),

    # TODO: figure out a reasonable value here
    transforms.CenterCrop(32)
])


wikiart = ds.pytorch( 
    transform={'images': transformation, 'labels': None},
    tensors=["images", "labels"],
    # decode_method={'images':'pil', 'labels': 'numpy'},
    shuffle=True,
    batch_size=32
)

Copying dataset: 100%|██████████| 1853/1853 [4:06:43<00:00 

./dataset\hub_activeloop_wiki-art loaded successfully.



c:\Users\ramon\Developer\.venv\lib\site-packages\deeplake\integrations\pytorch\common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [1]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3,n,kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n,n//2,kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n//2,n//2,kernel_size=3, padding=1)

        self.ch = 4 * 4 * (n//2)
        self.fc1 = nn.Linear(self.ch, 32)
        self.fc2 = nn.Linear(32, 27)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, self.ch)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out


def training_function(train_loader, model, loss_fn, optimizer, device, n_epochs):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            labels = labels.flatten() # this is needed because for some reason every label is in a seperate tensor
            print(labels)
            print(imgs.shape)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))


cnn32 = Net(32)

c:\Users\ramon\Developer\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.optim as optim

device = (torch.device('cuda') if torch.cuda.is_available()
                      else torch.device('cpu'))

print(f"Training on {device} \n\n")

training_function(
    train_loader = wikiart,
    model = cnn32.to(device=device),
    loss_fn = nn.NLLLoss(),
    optimizer = optim.SGD(cnn32.parameters(), lr=1e-2),
    device=device,
    n_epochs = 2,
)

Training on cuda 




NameError: name 'wikiart' is not defined